In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
from baselines import ZeroUniformOutputBaseline

from network.training_heloc import train_model

model = train_model(
    layers=[23,64,2],
    num_epochs=10,
    lr = 0.001,
    batch_size=16,
)[0]



100%|██████████| 10/10 [00:09<00:00,  1.08it/s]


Final metrics: 
Validation-Loss:  0.654
Validation-Accuracy:  66.3%
train-Loss:  0.553
train-Accuracy:  68.6%
# Parameters: 1666
NeuralNetwork(
  (model): Sequential(
    (0): Linear(in_features=23, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=2, bias=True)
  )
)


In [11]:
import torch
print(f"Prediction of zero baseline: {model.predict(torch.zeros(1, 23)).detach().numpy()})")

baseline_generator = ZeroUniformOutputBaseline(model)
print(f"Number of inputs: {baseline_generator.num_inputs}")
zero_uniform_output_baseline = baseline_generator.get_baseline()

/home/leon/programming/XAI-baselines/network/models.py:65: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return nn.functional.softmax(y).detach()
/home/leon/programming/XAI-baselines/network/models.py:68: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return nn.functional.softmax(y)


Prediction of zero baseline: [[0.5250058 0.4749942]])
Number of inputs: 23
autobaseline: tensor([ 0.0075,  0.0058,  0.0061,  0.0061,  0.0096,  0.0079,  0.0076,  0.0085,
         0.0089,  0.0079,  0.0051,  0.0073,  0.0067,  0.0089,  0.0078,  0.0061,
        -0.0441,  0.0084,  0.0074,  0.0086,  0.0086,  0.0053,  0.0067],
       grad_fn=<SqueezeBackward3>)
prediction: tensor([0.5006, 0.4994])


In [19]:
from data import HELOC

dataset = HELOC("validation")
dataset[0]

(tensor([0.8911, 0.2925, 0.1070, 0.3052, 0.4943, 0.3600, 0.3600, 1.0000, 0.0222,
         0.8889, 1.0000, 0.4037, 0.3571, 0.5046, 0.2727, 0.5152, 0.5152, 0.0798,
         0.5432, 0.3667, 0.5217, 0.4286, 0.4679]),
 tensor([0., 1.]))